# BitMEX Wallets in/outflow
Date: 5/20/20  
Author: Evan Azevedo  
Organization: Amberdata  

## Description
Exploring the question: observing the in/outflows from BitMEX wallets, is there some impact on price? 

## Research Question
Does the inflow of money into BitMEX wallets have any correlation with the price?

### Key information
* All wallet addresses start with BMEX (I think those are motor bikes?)
* BitMEX have a single daily withdraw. So these wallets can take a lot of deposits, but limited withdraw - similar to a High Yield Savings acct.